# Cascade Error Correction Protocol Demonstration
This notebook demonstrates the **Cascade Error Correction Protocol** used in Quantum Key Distribution (QKD). 

In QKD protocols like BB84, quantum transmission over a channel inevitably introduces noise (Quantum Bit Error Rate - QBER). Before a secure key can be used for encryption, Alice and Bob must perform **Information Reconciliation** to ensure their keys match exactly, while minimizing the information leaked to an eavesdropper.

### Key Concepts:
1. **Sifting**: Discarding bits where Alice and Bob used different bases.
2. **QBER Estimation**: Publicly revealing a small subset of bits to estimate the error rate.
3. **Cascade**: A multi-pass block-parity protocol that corrects bit-flip errors.

---
## 1. Environment Setup and Protocol Import
We begin by importing the necessary libraries and the `CascadeProtocol` implementation.

In [1]:
import numpy as np
import random
import math
import sys
import os

# Ensure the library path is added if needed
# sys.path.append(os.path.abspath('.'))

from ErrorCorrection.cascade import CascadeProtocol

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("Libraries imported successfully.")

Libraries imported successfully.


## 2. Simulating the Sifted Key Generation
In a real BB84 setup, the "sifted key" is what remains after Alice and Bob compare their bases and discard the bits where they didn't match.

We will simulate this by generating a random bit string for Alice and a noisy version for Bob.
We introduce errors with a probability $\epsilon$ representing the QBER.

In [2]:
def generate_keys(length, target_qber):
    # Alice generates a random bit string
    alice_key = np.random.randint(0, 2, length).tolist()
    
    # Bob starts with a copy of Alice's key
    bob_key = list(alice_key)
    
    # Introduce random bit-flip errors
    error_indices = np.random.choice(range(length), int(length * target_qber), replace=False)
    for idx in error_indices:
        bob_key[idx] = 1 - bob_key[idx]
        
    actual_qber = len(error_indices) / length
    return alice_key, bob_key, actual_qber

KEY_LENGTH = 1000
TARGET_QBER = 0.05

alice_sifted, bob_sifted, actual_qber = generate_keys(KEY_LENGTH, TARGET_QBER)

print(f"Generated Keys of length: {KEY_LENGTH}")
print(f"Actual QBER in generated keys: {actual_qber:.4%}")
print(f"First 100 bits of Alice: {alice_sifted[:100]}")
print(f"First 100 bits of Bob:   {bob_sifted[:100]}")

Generated Keys of length: 1000
Actual QBER in generated keys: 5.0000%
First 100 bits of Alice: [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
First 100 bits of Bob:   [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]


## 3. Estimating the Quantum Bit Error Rate (QBER)
Alice and Bob don't know the exact QBER of the entire key without revealing everything. Instead, they sample a small subset of the key to estimate the error rate. 

This estimate is crucial because the Cascade protocol uses it to determine the optimal block size for correction.
Higher QBER requires smaller initial block sizes.

Estimated QBER $\hat{\epsilon} = \frac{n_{errors}}{n_{sample}}$

In [3]:
def estimate_qber(alice_key, bob_key, sample_size=0.1):
    n = len(alice_key)
    k = int(n * sample_size)
    
    # Randomly select indices for the sample
    sample_indices = random.sample(range(n), k)
    
    errors = sum(1 for i in sample_indices if alice_key[i] != bob_key[i])
    estimated_qber = errors / k if k > 0 else 0
    
    return estimated_qber

SAMPLE_SIZE = 0.2
est_qber = estimate_qber(alice_sifted, bob_sifted, sample_size=SAMPLE_SIZE)

print(f"Sample Size: {SAMPLE_SIZE*100}%")
print(f"Estimated QBER: {est_qber:.4%}")
print(f"True QBER:      {actual_qber:.4%}")

Sample Size: 20.0%
Estimated QBER: 3.5000%
True QBER:      5.0000%


## 4. Applying the Cascade Error Correction Protocol
Now we run the **Cascade** protocol. 

The protocol works in multiple passes:
1. **Pass 1**: Key is divided into blocks of size $k \approx 0.73 / \hat{\epsilon}$. Parity of each block is compared.
2. **Binary Search**: If parities of a block don't match, Bob and Alice perform a binary search on that block to find the bit causing the error. Bob flips that bit.
3. **Cascading**: Once an error is found and flipped in Pass $i$, Bob checks if any blocks in previous passes ($<i$) that contained that bit now have an odd number of errors. If so, he recursively corrects them.
4. **Subsequent Passes**: The key is randomly shuffled, block size is doubled, and the process repeats.

We use the `CascadeProtocol` class imported earlier.

In [4]:
# Initialize protocol with 4 passes
protocol = CascadeProtocol(num_passes=4, verbose=True)

# Run the reconciliation
# Bob receives Alice's key and uses Alice's parity responses to correct his own key
corrected_key_bob, bits_revealed, total_corrected, channel_uses = protocol.run(
    alice_sifted, 
    bob_sifted, 
    qber=est_qber
)

print("\n--- RECONCILIATION COMPLETE ---")
print(f"Bits Revealed during parity checks: {bits_revealed}")
print(f"Total Bit Errors Corrected:       {total_corrected}")
print(f"Number of Channel Round-Trips:    {channel_uses}")

Starting Cascade Protocol with 4 passes.

--- Pass 1 ---
  Block size k: 20
  Blocks with odd parity errors: 14 / 50
  Correcting error in block 3...
  [BINARY] Starting binary search on block of size 20
    [BINARY] Parity match in left half. Error in right half (10-19). Recursing right.
    [BINARY] Parity mismatch in left half (10-14). Recursing left.
    [BINARY] Parity mismatch in left half (10-12). Recursing left.
    [BINARY] Parity mismatch in left half (10-11). Recursing left.
    [BINARY] Parity mismatch in left half (10-10). Recursing left.
  [BINARY] Found error at index: 70
  Correcting error in block 7...
  [BINARY] Starting binary search on block of size 20
    [BINARY] Parity mismatch in left half (0-9). Recursing left.
    [BINARY] Parity match in left half. Error in right half (5-9). Recursing right.
    [BINARY] Parity match in left half. Error in right half (8-9). Recursing right.
    [BINARY] Parity match in left half. Error in right half (9-9). Recursing right.
  

## 5. Verification and Key Comparison
Finally, we verify if Alice's and Bob's keys now match. 

We will display a comparison table for the first 25 bits, highlighting the status of each bit:
- **OK**: Alice and Bob matched from the start.
- **FIXED**: Bob had an error but Cascade corrected it.
- **ERROR**: Bob still has an error (Cascade failed to find it).
- **BROKE**: Bob had the correct bit but Cascade flipped it incorrectly (Rare but possible in complex scenarios).

In [5]:
def print_key_comparison(alice_key, bob_key_raw, bob_key_corrected, limit=25):
    print(f"{'Idx':<4} | {'Alice':<5} | {'Bob Raw':<7} | {'Bob Final':<9} | {'Status'}")
    print("-" * 50)
    
    for i in range(min(limit, len(alice_key))):
        a = alice_key[i]
        b_raw = bob_key_raw[i]
        b_final = bob_key_corrected[i]
        
        status = "OK"
        if a != b_raw and a == b_final: status = "FIXED"
        elif a != b_raw and a != b_final: status = "ERROR"
        elif a == b_raw and a != b_final: status = "BROKE"
        
        print(f"{i:<4} | {a:<5} | {b_raw:<7} | {b_final:<9} | {status}")

print_key_comparison(alice_sifted, bob_sifted, corrected_key_bob)

# Final check
final_match = (alice_sifted == corrected_key_bob)
print(f"\nFinal Keys Match: {final_match}")
if not final_match:
    remaining_errors = sum(1 for a, b in zip(alice_sifted, corrected_key_bob) if a != b)
    print(f"Remaining Errors: {remaining_errors}")

Idx  | Alice | Bob Raw | Bob Final | Status
--------------------------------------------------
0    | 0     | 0       | 0         | OK
1    | 1     | 1       | 1         | OK
2    | 0     | 0       | 0         | OK
3    | 0     | 0       | 0         | OK
4    | 0     | 0       | 0         | OK
5    | 1     | 1       | 1         | OK
6    | 0     | 0       | 0         | OK
7    | 0     | 0       | 0         | OK
8    | 0     | 0       | 0         | OK
9    | 1     | 1       | 1         | OK
10   | 0     | 0       | 0         | OK
11   | 0     | 0       | 0         | OK
12   | 0     | 0       | 0         | OK
13   | 0     | 0       | 0         | OK
14   | 1     | 1       | 1         | OK
15   | 0     | 0       | 0         | OK
16   | 1     | 1       | 1         | OK
17   | 1     | 1       | 1         | OK
18   | 1     | 1       | 1         | OK
19   | 0     | 0       | 0         | OK
20   | 1     | 0       | 1         | FIXED
21   | 0     | 1       | 0         | FIXED
22   | 1     | 1   

## 6. Analyzing Reconciliation Efficiency and Information Leakage
Reconciliation is not "free". Every parity bit Alice sends to Bob reveals 1 bit of information to the eavesdropper (Eve). We want to minimize this leakage.

### Shannon Limit:
The theoretical minimum number of bits that *must* be revealed to correct errors at QBER $\epsilon$ is given by the binary entropy:
$$H(\epsilon) = -\epsilon \log_2(\epsilon) - (1-\epsilon) \log_2(1-\epsilon)$$

### Reconciliation Efficiency ($\eta$):
The efficiency of a protocol is the ratio of actual bits revealed ($m$) to the theoretical minimum for $n$ bits:
$$\eta = \frac{m}{n \cdot H(\epsilon)}$$

An ideal protocol has $\eta = 1$. Most practical protocols like Cascade have $1.1 < \eta < 2.0$.

In [6]:
def binary_entropy(p):
    if p <= 0 or p >= 1:
        return 0
    return -p * math.log2(p) - (1-p) * math.log2(1-p)

h_eps = binary_entropy(actual_qber)
min_revealed = KEY_LENGTH * h_eps

efficiency = bits_revealed / min_revealed if min_revealed > 0 else 0

print(f"Actual QBER (\u03b5):          {actual_qber:.4%}")
print(f"Shannon Entropy H(\u03b5):     {h_eps:.4f} bits/bit")
print(f"Theoretical Min Leakage: {min_revealed:.2f} bits")
print(f"Actual Leakage (m):      {bits_revealed} bits")
print(f"Reconciliation Efficiency (\u03b7): {efficiency:.4f}")

if efficiency < 1.0:
    print("Warning: Efficiency < 1.0 is theoretically impossible for a perfect estimate. (Likely caused by statistical noise or small sample size.)")

Actual QBER (ε):          5.0000%
Shannon Entropy H(ε):     0.2864 bits/bit
Theoretical Min Leakage: 286.40 bits
Actual Leakage (m):      330 bits
Reconciliation Efficiency (η): 1.1522
